In [125]:
import torch
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
import PIL.Image
from torchvision.utils import make_grid
from utils import show

In [126]:
from data import RawWaterMeterDS
from utils import tuple_collate_fn, draw_bbox_with_tensor, object_detection_transform
from models.SingleObjectDetectionNetwork import fasterrcnn_mobilenet_v3_large_320_fpn

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [156]:
dataset = RawWaterMeterDS('train', root='WaterMeterDataset/ProcessedData', target_transform=object_detection_transform)
train_ds, valid_ds = data.random_split(dataset, [0.8, 0.2])

In [160]:
model = fasterrcnn_mobilenet_v3_large_320_fpn()
model.to(device)
dl = data.DataLoader(dataset, 6, shuffle=True, collate_fn=tuple_collate_fn)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

In [203]:
mean_loss = 0
model.train()
for _ in range(10):
    for idx, (X, y) in enumerate(dl):
        optimizer.zero_grad()
        X = list(X)
        for i in range(len(X)):
            X[i] = X[i].to(device)
        for j in range(len(y)):
            y[j]['boxes'] = y[j]['boxes'].to(device)
            y[j]['labels'] = y[j]['labels'].to(device)
        loss_dict = model(X, y)
        # print(loss_dict
        loss = sum(loss_dict.values())
        loss.backward()
        optimizer.step()
        mean_loss += loss.item()
        if (idx + 1) % 20 == 0:
            print(mean_loss / 20)
            mean_loss = 0

0.27946359291672707
0.20567648820579051
0.28954426646232606
0.21443540155887603
0.2515687167644501
0.23652320243418218
0.261049884557724
0.2619100596755743
0.31046064384281635
0.21465664766728879
0.257125736400485
0.2268220901489258
0.24808392785489558
0.25108129531145096
0.26036474816501143
0.23870575837790967
0.28292456530034543
0.20452821142971517
0.2258294366300106
0.20843336582183838
0.2262719888240099
0.24162195734679698
0.22437077276408673
0.24561519771814347
0.2953087896108627
0.23447807058691977
0.21404892057180405
0.2078272070735693
0.2045809779316187
0.22685796283185483
0.20439740754663943
0.23611570186913014
0.2504621375352144
0.18118622414767743
0.20143641382455826
0.20836824290454387
0.21421085000038148
0.19329186379909516
0.22759792432188988
0.22299696058034896


In [ ]:
# show detection effects in test dataset
test_ds = RawWaterMeterDS('test', root='WaterMeterDataset/ProcessedData')
model.eval()
image_idx = 411
img = draw_bbox_with_tensor(test_ds[image_idx], model(test_ds[image_idx].unsqueeze(0).to(device))[0]['boxes'])
show(make_grid(img))

In [280]:
# transform test images
with torch.no_grad():
    for idx in range(500):
        img = test_ds[idx]
        pred = model(img.unsqueeze(0).to(device))[0]['boxes'].to(torch.int)
        # check if bbox exists
        bbox = pred[0] if len(pred) != 0 else torch.tensor([0, 0, 100, 100])
        patch = img[:, bbox[1]: bbox[3], bbox[0]: bbox[2]]
        patch = transforms.ToPILImage()(patch)
        patch.save(f'WaterMeterDataset/ProcessedData/test_imgs_seg/test_seg_{idx + 1}.jpg')

In [ ]:
# transform train images
for idx in range(1000):
    img = dataset[idx][0]
    bbox = dataset[idx][1]['boxes'][0]
    patch = img[:, max(bbox[1], torch.tensor([0])): bbox[3], max(bbox[0], torch.tensor([0])): bbox[2]]
    patch = transforms.ToPILImage()(patch)
    patch.save(f'WaterMeterDataset/ProcessedData/train_imgs_seg/train_seg_{idx + 1}.jpg')